# Import modules

In [1]:
#Import Modules
import json
import sys
import nbib
import pandas as pd
import requests as rq
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pyperclip
from datetime import datetime
import os
from bs4 import BeautifulSoup

from Bio import Entrez
Entrez.email = "your_email@example.com"

# Import dataset

In [2]:
#import dataset
df_gwas = pd.read_table('gwas_catalog_data/gwas_catalog_v1.0.2-associations_e109_r2023-03-27.tsv')
df_gwas[['PUBMEDID','SNP_ID_CURRENT']] = df_gwas[['PUBMEDID','SNP_ID_CURRENT']].astype(str)
print('genes: ', len(df_gwas['MAPPED_GENE'].drop_duplicates()))
print('studies: ', len(df_gwas['STUDY'].drop_duplicates()))
print('rsid: ', len(df_gwas['SNP_ID_CURRENT'].drop_duplicates()))
df_gwas.columns

C:\Users\giova\AppData\Local\Temp\ipykernel_25668\3332296947.py:2: DtypeWarning: Columns (9,11,12,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gwas = pd.read_table('gwas_catalog_data/gwas_catalog_v1.0.2-associations_e109_r2023-03-27.tsv')


genes:  44391
studies:  5537
rsid:  275806


Index(['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
       'LINK', 'STUDY', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE',
       'REPLICATION SAMPLE SIZE', 'REGION', 'CHR_ID', 'CHR_POS',
       'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'MERGED', 'SNP_ID_CURRENT', 'CONTEXT', 'INTERGENIC',
       'RISK ALLELE FREQUENCY', 'P-VALUE', 'PVALUE_MLOG', 'P-VALUE (TEXT)',
       'OR or BETA', '95% CI (TEXT)', 'PLATFORM [SNPS PASSING QC]', 'CNV',
       'MAPPED_TRAIT', 'MAPPED_TRAIT_URI', 'STUDY ACCESSION',
       'GENOTYPING TECHNOLOGY'],
      dtype='object')

In [7]:
#df_gwas

In [3]:
#display selected columns
df_gwas['STRONGEST SNP-RISK ALLELE'].drop_duplicates()
df_gwas[['MAPPED_GENE','DISEASE/TRAIT','MAPPED_TRAIT','SNP_ID_CURRENT','STRONGEST SNP-RISK ALLELE','RISK ALLELE FREQUENCY']].drop_duplicates()

,MAPPED_GENE,DISEASE/TRAIT,MAPPED_TRAIT,SNP_ID_CURRENT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
0,MTHFR,Homocysteine levels,homocysteine measurement,1801133,rs1801133-A,0.34
1,MTR,Homocysteine levels,homocysteine measurement,2275565,rs2275565-T,0.21
2,CPS1,Homocysteine levels,homocysteine measurement,1047891,rs7422339-A,0.33
3,EEF1A1P42 - MMUT,Homocysteine levels,homocysteine measurement,9369898,rs9369898-A,0.62
4,NOX4,Homocysteine levels,homocysteine measurement,7130284,rs7130284-T,0.07
...,...,...,...,...,...,...
496335,CCDC34,Height,body height,12419259.0,rs12419259-C,0.3185
496336,MPPED2,Height,body height,474376.0,rs474376-C,0.2378
496337,MPPED2,Height,body height,16920925.0,rs16920925-C,0.106
496338,HSD17B12,Height,body height,11037654.0,rs11037654-A,0.2181


In [4]:
print('SNPs:',len(df_gwas.SNPS.drop_duplicates()))
print('DISEASE/TRAIT:',len(df_gwas['DISEASE/TRAIT'].drop_duplicates()))
print('MAPPED_TRAIT:',len(df_gwas['MAPPED_TRAIT'].drop_duplicates()))
df_gwas['MAPPED_TRAIT'].value_counts()

SNPs: 267372
DISEASE/TRAIT: 21399
MAPPED_TRAIT: 7690


body height                                                                 22757
body mass index                                                             11498
blood protein measurement                                                    8205
BMI-adjusted waist-hip ratio                                                 7796
triglyceride measurement                                                     7148
                                                                            ...  
response to TNF antagonist, arthritis disease activity score measurement        1
self rated health, response to TNF antagonist                                   1
bromodomain-containing protein 1 measurement                                    1
response to TNF antagonist, blood sedimentation                                 1
protein measurement, C-reactive protein measurement                             1
Name: MAPPED_TRAIT, Length: 7689, dtype: int64

# Filter dataset

 #Workflow------------------------

in STRONGEST SNP-RISK ALLEL drop "?"
filter for your rsids
--> I need the Merged data from GRPM-X:
      - [gene, rsids, pmids, mesh(?)]
merge tables on rsids
comparare GENE-MESH vs
get the STRONGEST SNP-RISK ALLELE

Column Selection Mode = Alt+Maiusc+Ins
#https://www.jetbrains.com/help/rider/Multicursor.html#add-and-remove-carets

In [9]:
# Drop no risk allele:
mask = df_gwas['STRONGEST SNP-RISK ALLELE'].str.contains("\?")
df_gwas_drop = df_gwas[-mask].reset_index(drop=True)

# Drop complementary base allele (risk allele freq missing)
df_gwas_drop_nonan = df_gwas_drop.dropna(subset=['RISK ALLELE FREQUENCY'],axis=0).reset_index(drop=True)
print('Drop no risk allele:')
print('SNPs:',len(df_gwas_drop_nonan.SNPS.drop_duplicates()))
print('DISEASE/TRAIT:',len(df_gwas_drop_nonan['DISEASE/TRAIT'].drop_duplicates()))
print('MAPPED_TRAIT:',len(df_gwas_drop_nonan['MAPPED_TRAIT'].drop_duplicates()))


Drop no risk allele:
SNPs: 195998
DISEASE/TRAIT: 16820
MAPPED_TRAIT: 6294


Full dataset:
SNPs: 267372
DISEASE/TRAIT: 21399
MAPPED_TRAIT: 7690

## LOOKUP FOR SINGLE RSID

In [6]:
# LOOKUP FOR SINGLE RSID
rsid_mask = df_gwas_drop_nonan['SNPS'].str.contains('rs1421085')
df_gwas_drop_nonan_rsid = df_gwas_drop_nonan[rsid_mask]
df_gwas_drop_nonan_rsid[['MAPPED_GENE','DISEASE/TRAIT','MAPPED_TRAIT','SNP_ID_CURRENT','STRONGEST SNP-RISK ALLELE','RISK ALLELE FREQUENCY']].drop_duplicates()
df_gwas_drop_nonan_rsid.value_counts('DISEASE/TRAIT')
df_gwas_drop_nonan_rsid.value_counts('STRONGEST SNP-RISK ALLELE')

STRONGEST SNP-RISK ALLELE
rs1421085-C    31
rs1421085-T    16
dtype: int64

In [7]:
# Display and to clipboard
df_gwas_drop_nonan[['MAPPED_GENE', 'DISEASE/TRAIT', 'SNPS','STRONGEST SNP-RISK ALLELE', 'RISK ALLELE FREQUENCY']].drop_duplicates()#to_clipboard()

,MAPPED_GENE,DISEASE/TRAIT,SNPS,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
0,MTHFR,Homocysteine levels,rs1801133,rs1801133-A,0.34
1,MTR,Homocysteine levels,rs2275565,rs2275565-T,0.21
2,CPS1,Homocysteine levels,rs7422339,rs7422339-A,0.33
3,EEF1A1P42 - MMUT,Homocysteine levels,rs9369898,rs9369898-A,0.62
4,NOX4,Homocysteine levels,rs7130284,rs7130284-T,0.07
...,...,...,...,...,...
376528,CCDC34,Height,rs12419259,rs12419259-C,0.3185
376529,MPPED2,Height,rs474376,rs474376-C,0.2378
376530,MPPED2,Height,rs16920925,rs16920925-C,0.106
376531,HSD17B12,Height,rs11037654,rs11037654-A,0.2181


In [28]:
df_gwas_drop_nonan_rsid[['MAPPED_GENE', 'DISEASE/TRAIT', 'SNPS','STRONGEST SNP-RISK ALLELE', 'RISK ALLELE FREQUENCY']].drop_duplicates()#.to_clipboard()

,MAPPED_GENE,DISEASE/TRAIT,SNPS,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
3050,FTO,Dietary macronutrient intake,rs1421085,rs1421085-C,0.42
4787,FTO,Obesity,rs1421085,rs1421085-C,0.4
4789,FTO,Obesity,rs1421085,rs1421085-C,0.41
6904,FTO,Obesity (early onset extreme),rs1421085,rs1421085-C,0.41
12774,FTO,Childhood body mass index,rs1421085,rs1421085-C,0.41
21154,FTO,Type 2 diabetes,rs1421085,rs1421085-C,0.415
41246,FTO,Age at adiposity rebound,rs1421085,rs1421085-C,0.25
41753,FTO,Fasting insulin,rs1421085,rs1421085-T,0.578326
49116,FTO,Body mass index,rs1421085,rs1421085-C,NR
49809,FTO,Pulse pressure,rs1421085,rs1421085-T,0.7995


# Merge GWAS and GRPMX data

In [3]:
# load my GRPMx Data
directory = 'grpm_survey_pcg_'+'nutri'
df_grpmx = pd.read_csv(directory+'/grpmx_filtered_output.csv', index_col=0)
df_grpmx_repo = pd.read_csv(directory+'/GRPMX_report_int.csv')

#filter for 0.95 quantile
df_grpmx_repo_95 = df_grpmx_repo[df_grpmx_repo.interest_index >= df_grpmx_repo.interest_index.quantile(0.95)]
df_grpmx_95 = df_grpmx[df_grpmx.gene.isin(df_grpmx_repo_95.gene)]
print('df_grpmx_95 genes:', df_grpmx_95.gene.nunique())
df_grpmx_95

df_grpmx_95 genes: 776


,gene,rsid,pmids,mesh
0,MT-ND1,rs111033358,32387288,Cardiovascular Diseases
1,MT-ND1,rs2854128,32387288,Cardiovascular Diseases
2,MT-ND1,rs111033358,33468709,"Diabetes Mellitus, Type 2"
3,MT-ND1,rs111033358,33468709,Glucose
4,MT-ND1,rs111033358,33468709,Insulin
...,...,...,...,...
319,CXCL8,rs572157399,23769592,Inflammation
320,CXCL8,rs572157399,23769592,Obesity
321,CXCL8,rs758228010,27992599,Oxidative Stress
322,CXCL8,rs758228010,27992599,Proteolysis


In [4]:
#threshold
df_grpmx_t = df_grpmx_95

## ADD gene-interest index as common sorting handle
small_dummy = df_grpmx_repo[['gene','interest_index']]
df_grpmx_int = pd.merge(df_grpmx, small_dummy, left_on='gene', right_on='gene')
df_grpmx_95_int = pd.merge(df_grpmx_95, small_dummy, left_on='gene', right_on='gene')

print('GRPMX threshold Statistics:')
print('genes:', df_grpmx_t.gene.nunique())
print('rsid:', df_grpmx_t.rsid.nunique())
print('mesh:', df_grpmx_t.mesh.nunique())
df_grpmx_int

GRPMX threshold Statistics:
genes: 776
rsid: 28138
mesh: 400


,gene,rsid,pmids,mesh,interest_index
0,MT-ND1,rs111033358,32387288,Cardiovascular Diseases,0.00566
1,MT-ND1,rs2854128,32387288,Cardiovascular Diseases,0.00566
2,MT-ND1,rs111033358,33468709,"Diabetes Mellitus, Type 2",0.00566
3,MT-ND1,rs111033358,33468709,Glucose,0.00566
4,MT-ND1,rs111033358,33468709,Insulin,0.00566
...,...,...,...,...,...
416314,EVC,rs6414624,29273463,Blood Glucose,0.00002
416315,EVC,rs6414624,29273463,"Diabetes Mellitus, Type 2",0.00002
416316,TMC3,rs150843673,29691411,"Diabetes Mellitus, Type 2",0.00045
416317,TMC3,rs150843673,29691411,Obesity,0.00045


In [5]:
complete_merge = False
if complete_merge == True:
    #common handle sort
    df_grpmx_int = df_grpmx_int.sort_values(by=['interest_index','rsid','mesh'], ascending =False).reset_index(drop=True)
else:
    df_grpmx_95_int = df_grpmx_95_int.sort_values(by=['interest_index','rsid','mesh'], ascending =False).reset_index(drop=True)


In [6]:
if complete_merge == True:
    # Merge two df on rsid:
    df_merged = pd.merge(df_grpmx_int, df_gwas, left_on='rsid', right_on='SNPS')
    df_merged[['pmids','PUBMEDID']] = df_merged[['pmids','PUBMEDID']].astype(str)

    df_merged_drop = pd.merge(df_grpmx_int, df_gwas_drop_nonan, left_on='rsid', right_on='SNPS')
    df_merged_drop[['pmids','PUBMEDID']] = df_merged_drop[['pmids','PUBMEDID']].astype(str)

In [7]:
if complete_merge == True:
    print('complete grpmx merge')
    print(len(df_merged_drop), len(df_merged))
    df_merged_drop[['gene','rsid','mesh','DISEASE/TRAIT','STRONGEST SNP-RISK ALLELE']].drop_duplicates()
    print(df_merged_drop.rsid.nunique(), 'rsid')
    print(df_merged_drop.mesh.nunique(), 'mesh')
    print(df_merged_drop['DISEASE/TRAIT'].nunique(), 'DISEASE/TRAIT')
    print(df_merged_drop['STRONGEST SNP-RISK ALLELE'].nunique(), 'STRONGEST SNP-RISK ALLELE')

    df_merged_drop[['gene','rsid','mesh','DISEASE/TRAIT','STRONGEST SNP-RISK ALLELE']].drop_duplicates()

In [8]:
merge_nonan = True
merge_both = True

if complete_merge == False:
    # Merge two df on rsid:
    if merge_nonan == False:
        df_merged_95 = pd.merge(df_grpmx_95_int, df_gwas, left_on='rsid', right_on='SNPS')
        df_merged_95[['pmids','PUBMEDID']] = df_merged_95[['pmids','PUBMEDID']].astype(str)
    if merge_nonan == True:
        df_merged_95_drop = pd.merge(df_grpmx_95_int, df_gwas_drop_nonan, left_on='rsid', right_on='SNPS')
        df_merged_95_drop[['pmids','PUBMEDID']] = df_merged_95_drop[['pmids','PUBMEDID']].astype(str)
    if merge_both == True:
        df_merged_95 = pd.merge(df_grpmx_95_int, df_gwas, left_on='rsid', right_on='SNPS')
        df_merged_95[['pmids','PUBMEDID']] = df_merged_95[['pmids','PUBMEDID']].astype(str)
print('genes merged:', df_merged_95_drop.gene.nunique())
df_merged_95_drop

NameError: name 'df_gwas_drop_nonan' is not defined

In [20]:
print('threshold (q0.95) grpmx merge')
print('full 95 merge pmids:',df_merged_95.pmids.nunique(), 'drop95 pmids:',df_merged_95_drop.pmids.nunique())
#df_merged_95_drop[['gene','rsid','mesh','DISEASE/TRAIT','STRONGEST SNP-RISK ALLELE']].drop_duplicates()
print('grpm merged with gwas stats:')
print(' ',df_merged_95_drop.gene.nunique(), 'gene')
print(' ',df_merged_95_drop.pmids.nunique(), 'pmid')
print(' ',df_merged_95_drop.mesh.nunique(), 'mesh')
print(' ',df_merged_95_drop.rsid.nunique(), 'rsid')
print(' ',df_merged_95_drop['DISEASE/TRAIT'].nunique(), 'DISEASE/TRAIT')
print(' ',df_merged_95_drop['STRONGEST SNP-RISK ALLELE'].nunique(), 'STRONGEST SNP-RISK ALLELE')

threshold (q0.95) grpmx merge
full 95 merge pmids: 26337 drop95 pmids: 25467
grpm merged with gwas stats:
  644 gene
  25467 pmid
  367 mesh
  4086 rsid
  4795 DISEASE/TRAIT
  5798 STRONGEST SNP-RISK ALLELE


In [96]:
df_merged_less = df_merged.drop(columns=['DATE ADDED TO CATALOG','MERGED','SNP_ID_CURRENT'])
len(df_merged_less['STUDY'].drop_duplicates())
print('GWAS TRAIT count con entire grpmx')
df_merged_less.value_counts('DISEASE/TRAIT')

GWAS TRAIT count con entire grpmx


DISEASE/TRAIT
Type 2 diabetes                                                          273228
Body mass index                                                          130060
Total cholesterol levels                                                 117161
Serum metabolite levels                                                   90610
Low density lipoprotein cholesterol levels                                76152
                                                                          ...  
Serum levels of protein ADAM11                                                1
Serum levels of protein AKR1B1                                                1
CDGSH iron-sulfur domain-containing protein 1 levels (CISD1.7745.3.3)         1
Serum levels of protein C1orf56                                               1
Monocyte chemoattractant protein-1 levels                                     1
Length: 5631, dtype: int64

In [97]:
print('GWAS TRAIT count (nonan) con entire grpmx')
df_merged_drop_less = df_merged_drop.drop(columns=['DATE ADDED TO CATALOG','MERGED','SNP_ID_CURRENT'])
len(df_merged_less['STUDY'].drop_duplicates())
df_merged_drop_less.value_counts('DISEASE/TRAIT')

GWAS TRAIT count (nonan) con entire grpmx


DISEASE/TRAIT
Type 2 diabetes                                                                 251974
Body mass index                                                                 104366
Total cholesterol levels                                                         91208
Triglycerides                                                                    57623
Low density lipoprotein cholesterol levels                                       52892
                                                                                 ...  
Serum levels of protein LRIT3                                                        1
COVID-19 (hospitalized vs tested, not hospitalized)                                  1
Serum levels of protein IFIT2                                                        1
Serum levels of protein GSTM1                                                        1
Pulse pressure x alcohol consumption (light vs heavy) interaction (2df test)         1
Length: 4875, dtype: int64

In [37]:
#rename columns:
df_merged_drop = df_merged_drop.rename(columns={'gene':'LITVAR GENE', 'rsid':'LIVAR RSID', 'pmids':'LITVAR PMID','mesh':'PUBMED MESH'})
df_merged_drop_nonan = df_merged_drop

In [48]:
#df_merged_drop_nonan.to_csv('df_merged_drop_nonan.csv') # diversi GB!!!!!!!!!!!!!!!!
df_usage = df_merged.memory_usage()/1048576
df_usage.sum()

1843.2411651611328

In [103]:
df_merged_drop[['LIVAR RSID','LITVAR GENE']].drop_duplicates()

,LIVAR RSID,LITVAR GENE
0,rs1421085,FTO
34848,rs3751812,FTO
37004,rs56094641,FTO
37283,rs8050136,FTO
54143,rs9939609,FTO
...,...,...
4684800,rs1045475,ADCY9
4684801,rs227584,HROB
4684825,rs2073438,ALOX12
4684843,rs434473,ALOX12


#### Define Mother Dataframe:

In [49]:
df_merged_drop

,LITVAR GENE,LIVAR RSID,LITVAR PMID,PUBMED MESH,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,...,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
0,FTO,rs1421085,34556834,Eating,2013-09-17,23636237,Tanaka T,2013-05-01,Am J Clin Nutr,www.ncbi.nlm.nih.gov/pubmed/23636237,...,9.000000,(Protein),0.1000,[0.061-0.139] % increase,"Affymetrix, Illumina [2500000] (imputed)",N,energy intake,http://www.ebi.ac.uk/efo/EFO_0003939,GCST001989,Genome-wide genotyping array
1,FTO,rs1421085,34556834,Eating,2009-03-03,19151714,Meyre D,2009-01-18,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/19151714,...,28.000000,NaN,NaN,NaN,Illumina [308846],N,obesity,http://www.ebi.ac.uk/efo/EFO_0001073,GCST000317,Genome-wide genotyping array
2,FTO,rs1421085,34556834,Eating,2009-03-03,19151714,Meyre D,2009-01-18,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/19151714,...,17.154902,(children),1.3900,[1.27-1.51],Illumina [308846],N,obesity,http://www.ebi.ac.uk/efo/EFO_0001073,GCST000317,Genome-wide genotyping array
3,FTO,rs1421085,34556834,Eating,2009-03-03,19151714,Meyre D,2009-01-18,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/19151714,...,12.301030,(adults),1.2500,[1.10-1.40],Illumina [308846],N,obesity,http://www.ebi.ac.uk/efo/EFO_0001073,GCST000317,Genome-wide genotyping array
4,FTO,rs1421085,34556834,Eating,2013-10-24,23563609,Wheeler E,2013-04-07,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/23563609,...,27.522879,NaN,1.4400,[1.35-1.54],Affymetrix [~ 2000000] (imputed),N,obesity,http://www.ebi.ac.uk/efo/EFO_0001073,GCST001957,Genome-wide genotyping array
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639187,ALOX12,rs434473,20697415,Obesity,2023-02-09,36224396,Yengo L,2022-10-12,Nature,www.ncbi.nlm.nih.gov/pubmed/36224396,...,7.221849,NaN,0.0034,[0.0022-0.0046] unit increase,"Affymetrix, Illumina [1385132] (imputed)",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST90245848,Genome-wide genotyping array
4639188,ALOX12,rs312462,20697415,Body Fat Distribution,2023-02-09,36224396,Yengo L,2022-10-12,Nature,www.ncbi.nlm.nih.gov/pubmed/36224396,...,6.522879,NaN,0.0063,[0.0039-0.0087] unit increase,"Affymetrix, Illumina [1385132] (imputed)",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST90245848,Genome-wide genotyping array
4639189,ALOX12,rs312462,20697415,Body Mass Index,2023-02-09,36224396,Yengo L,2022-10-12,Nature,www.ncbi.nlm.nih.gov/pubmed/36224396,...,6.522879,NaN,0.0063,[0.0039-0.0087] unit increase,"Affymetrix, Illumina [1385132] (imputed)",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST90245848,Genome-wide genotyping array
4639190,ALOX12,rs312462,20697415,Bone Density,2023-02-09,36224396,Yengo L,2022-10-12,Nature,www.ncbi.nlm.nih.gov/pubmed/36224396,...,6.522879,NaN,0.0063,[0.0039-0.0087] unit increase,"Affymetrix, Illumina [1385132] (imputed)",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST90245848,Genome-wide genotyping array


In [51]:
# LOOKUP FOR SINGLE RSID
rsid_mask = df_merged_drop_nonan['LIVAR RSID'].str.contains('rs1421085')
df_merged_drop_nonan_rsid = df_merged_drop_nonan[rsid_mask]

In [56]:
# Display and to clipboard
df_merged_drop[['LITVAR GENE','LIVAR RSID','MAPPED_GENE','PUBMED MESH', 'DISEASE/TRAIT', 'STRONGEST SNP-RISK ALLELE', 'RISK ALLELE FREQUENCY']].drop_duplicates()
df_merged_drop_nonan_rsid[['LITVAR GENE','LIVAR RSID','MAPPED_GENE','PUBMED MESH', 'DISEASE/TRAIT', 'STRONGEST SNP-RISK ALLELE', 'RISK ALLELE FREQUENCY']].drop_duplicates()#.to_clipboard()

,LITVAR GENE,LIVAR RSID,MAPPED_GENE,PUBMED MESH,DISEASE/TRAIT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
0,FTO,rs1421085,FTO,Eating,Dietary macronutrient intake,rs1421085-C,0.42
1,FTO,rs1421085,FTO,Eating,Obesity,rs1421085-C,0.4
3,FTO,rs1421085,FTO,Eating,Obesity,rs1421085-C,0.41
4,FTO,rs1421085,FTO,Eating,Obesity (early onset extreme),rs1421085-C,0.41
6,FTO,rs1421085,FTO,Eating,Childhood body mass index,rs1421085-C,0.41
...,...,...,...,...,...,...,...
33176,FTO,rs1421085,FTO,Inflammation,Abdominal adipose tissue volumes to gluteofemo...,rs1421085-T,0.603
33177,FTO,rs1421085,FTO,Inflammation,C-reactive protein levels (MTAG),rs1421085-T,NR
33178,FTO,rs1421085,FTO,Inflammation,Mean corpuscular hemoglobin,rs1421085-C,NR
33179,FTO,rs1421085,FTO,Inflammation,Drinks per week,rs1421085-C,0.38


In [59]:
df_merged_drop_nonan_rsid['PUBMED MESH'].drop_duplicates()

0                                     Eating
47                                   Obesity
94                                Overweight
141                        Pediatric Obesity
188                           Adipose Tissue
                        ...                 
25286                    Glucose Intolerance
26367                            Adiponectin
29375    Insulin Receptor Substrate Proteins
31255                  Adipose Tissue, Beige
33135                           Inflammation
Name: PUBMED MESH, Length: 103, dtype: object

#### Get risk Allele list and use it to filter mother table

In [69]:
df_merged_drop_nonan.value_counts('SNPS')

SNPS
rs1260326    558992
rs429358     308315
rs7412       282736
rs7903146    268226
rs964184     249375
              ...  
rs2053163         1
rs204995          1
rs204994          1
rs2034899         1
rs2830088         1
Length: 4376, dtype: int64

In [66]:
#df_merged_drop_rsid_nonan[['STRONGEST SNP-RISK ALLELE','PUBMED MESH']].groupby('STRONGEST SNP-RISK ALLELE').describe().reset_index()
rsid_mask = df_merged_drop_nonan['LIVAR RSID'].str.contains('rs1421085')
df_merged_drop_nonan_rsid = df_merged_drop_nonan[rsid_mask]
df_merged_drop_nonan_rsid[['LITVAR GENE','LIVAR RSID','MAPPED_GENE','PUBMED MESH', 'DISEASE/TRAIT', 'STRONGEST SNP-RISK ALLELE', 'RISK ALLELE FREQUENCY']].drop_duplicates()
type(df_merged_drop_nonan_rsid['STRONGEST SNP-RISK ALLELE'].value_counts())#.head(1))
risk_allele = df_merged_drop_nonan_rsid['STRONGEST SNP-RISK ALLELE'].value_counts()#.index[0]
risk_allele

rs1421085-C    22506
rs1421085-T    11616
Name: STRONGEST SNP-RISK ALLELE, dtype: int64

In [54]:
#---> creare una lista programmaticamente di tutti i 'risk allele' by count and use it to filter mother dataframe with isin module!

# get all rsid list
rsid_list = df_merged_drop_nonan['LIVAR RSID'].drop_duplicates().to_list()
len(rsid_list)

4376

In [148]:
# risk allele pickup (part1)
time_start = datetime.now()
risk_allele_list = []
for i in rsid_list[:2000]:
    rsid_mask = df_merged_drop_nonan['LIVAR RSID'].str.contains(i)
    df_merged_drop_nonan_rsid = df_merged_drop_nonan[rsid_mask]
    risk_allele = df_merged_drop_nonan_rsid['STRONGEST SNP-RISK ALLELE'].value_counts().index[0]
    risk_allele_list.append(risk_allele)
    #print(str(risk_allele))
finish_start = datetime.now()
pd.Series(risk_allele_list).to_csv('risk_allele_list_0-2000.csv')

In [151]:
# risk allele pickup (part2)
for i in rsid_list[2000:]:
    rsid_mask = df_merged_drop_nonan['LIVAR RSID'].str.contains(i)
    df_merged_drop_nonan_rsid = df_merged_drop_nonan[rsid_mask]
    risk_allele = df_merged_drop_nonan_rsid['STRONGEST SNP-RISK ALLELE'].value_counts().index[0]
    risk_allele_list.append(risk_allele)
    #print(str(risk_allele))
finish_start = datetime.now()
print(finish_start - time_start)
pd.Series(risk_allele_list).to_csv('risk_allele_list_0-2000.csv')

In [55]:
# import back risk allele list

risk_allele_df = pd.read_csv('risk_allele_list_4376.csv', index_col=0)
risk_allele_list = risk_allele_df['0'].to_list()
risk_allele_list
# --> ora filtrare Mother Df per i risk allele and ...BAM!
risk_allele_df

,0
0,rs1421085-C
1,rs3751812-T
2,rs56094641-G
3,rs8050136-A
4,rs9939609-A
...,...
4371,rs2444217-A
4372,rs1045475-A
4373,rs227584-T
4374,rs434473-G


,LITVAR GENE,LIVAR RSID,MAPPED_GENE,PUBMED MESH,DISEASE/TRAIT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
0,FTO,rs1421085,FTO,Eating,Dietary macronutrient intake,rs1421085-C,0.42
1,FTO,rs1421085,FTO,Eating,Obesity,rs1421085-C,0.4
3,FTO,rs1421085,FTO,Eating,Obesity,rs1421085-C,0.41
4,FTO,rs1421085,FTO,Eating,Obesity (early onset extreme),rs1421085-C,0.41
6,FTO,rs1421085,FTO,Eating,Childhood body mass index,rs1421085-C,0.41
...,...,...,...,...,...,...,...
33176,FTO,rs1421085,FTO,Inflammation,Abdominal adipose tissue volumes to gluteofemo...,rs1421085-T,0.603
33177,FTO,rs1421085,FTO,Inflammation,C-reactive protein levels (MTAG),rs1421085-T,NR
33178,FTO,rs1421085,FTO,Inflammation,Mean corpuscular hemoglobin,rs1421085-C,NR
33179,FTO,rs1421085,FTO,Inflammation,Drinks per week,rs1421085-C,0.38


In [78]:
# genera la merged  with GRPMX dropped!
#df_merged_drop_nonan = pd.read_csv('df_merged_drop_nonan.csv', index_col=0)

df_merged_drop_less_gwa = df_merged_drop_nonan[['MAPPED_GENE','SNPS', 'DISEASE/TRAIT', 'STRONGEST SNP-RISK ALLELE', 'RISK ALLELE FREQUENCY']].drop_duplicates()
#df_merged_drop_less_gwa.to_csv('df_merged_drop_less_gwa.csv')#.SNPS.drop_duplicates()
df_merged_drop_less_gwa.SNPS.drop_duplicates().sample(10)

4613640    rs12185242
1648018         rs305
4393887    rs11556505
182715     rs17037390
4635623     rs3922844
4610209      rs798943
4377163     rs3734626
4626517    rs11086054
2358892     rs2393776
4602750      rs643531
Name: SNPS, dtype: object

In [82]:
type(df_merged_drop_nonan_rsid['STRONGEST SNP-RISK ALLELE'].value_counts())#.head(1))
risk_allele = df_merged_drop_nonan_rsid['STRONGEST SNP-RISK ALLELE'].value_counts()#.index[0]
risk_allele

rs62033400-G    66
rs62033400-A    11
Name: STRONGEST SNP-RISK ALLELE, dtype: int64

In [0]:
#qualti sono monorischio??
rsid_list = df_merged_drop_less_gwa.SNPS.drop_duplicates().to_list()
time_start = datetime.now()
monorisk_list = []

for i in rsid_list:
    rsid_mask = df_merged_drop_less_gwa['SNPS'].str.contains(i)
    df_merged_drop_less_gwa_rsid = df_merged_drop_less_gwa[rsid_mask].drop_duplicates()
    risk_allele = df_merged_drop_less_gwa_rsid['STRONGEST SNP-RISK ALLELE'].value_counts()#.index[0]
    if len(risk_allele)==1:
            monorisk = risk_allele.index[0]
            monorisk_list.append(monorisk)
            print(risk_allele)

In [229]:
#print(risk_allele)
pd.Series(monorisk_list)#.to_csv('monorisk_list.csv')
#len(rsid_list)

0        rs3751812-T
1        rs9939609-A
2       rs62048402-A
3        rs9939973-A
4        rs9927317-G
            ...     
2875     rs2531992-A
2876     rs2444217-A
2877     rs1045475-A
2878      rs434473-G
2879      rs312462-A
Length: 2880, dtype: object

In [102]:
# risk allele pickup: ricerca dei valori di conteggio ambigui------------------

rsid_list = df_merged_drop_less_gwa.SNPS.drop_duplicates().to_list()
time_start = datetime.now()
ambiguity_list = []

for i in rsid_list:
    rsid_mask = df_merged_drop_less_gwa['SNPS'].str.contains(i)
    df_merged_drop_less_gwa_rsid = df_merged_drop_less_gwa[rsid_mask].drop_duplicates()
    risk_allele = df_merged_drop_less_gwa_rsid['STRONGEST SNP-RISK ALLELE'].value_counts()#.index[0]
    if len(risk_allele)>1:
        if risk_allele[0] == risk_allele[1]:
            ambiguity = risk_allele.index[0]
            ambiguity_list.append(ambiguity)
            print(risk_allele)

print(risk_allele)
ambiguity_list

rs7185735-G    2
rs7185735-A    2
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs10852521-A    1
rs10852521-C    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs11075994-A    1
rs11075994-G    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs2072518-A    1
rs2072518-G    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs9651118-A    1
rs9651118-T    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs1010023-C    1
rs1010023-G    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs4823173-C    1
rs4823173-T    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs2072906-C    1
rs2072906-A    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs11128603-A    1
rs11128603-G    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs7626560-C    1
rs7626560-T    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs11709077-G    3
rs11709077-A    3
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs780095-G    1
rs780095-A    1
Name: STRONGEST SNP-RISK ALLELE, dtype: int64
rs780096-C    1
rs780096-G    1
Na

['rs7185735-G',
 'rs10852521-A',
 'rs11075994-A',
 'rs2072518-A',
 'rs9651118-A',
 'rs1010023-C',
 'rs4823173-C',
 'rs2072906-C',
 'rs11128603-A',
 'rs7626560-C',
 'rs11709077-G',
 'rs780095-G',
 'rs780096-C',
 'rs2282679-T',
 'rs689-T',
 'rs3842753-G',
 'rs2681472-A',
 'rs271-A',
 'rs12679834-T',
 'rs3735964-G',
 'rs6931514-G',
 'rs35612982-C',
 'rs1800795-C',
 'rs17248720-C',
 'rs4362-C',
 'rs4344-A',
 'rs4329-G',
 'rs4305-A',
 'rs11705701-A',
 'rs7651090-A',
 'rs73175572-A',
 'rs11927381-C',
 'rs12708980-C',
 'rs12713559-G',
 'rs679899-G',
 'rs1799964-C',
 'rs2134223-G',
 'rs2066847-C',
 'rs174556-T',
 'rs261332-G',
 'rs573922-G',
 'rs8192701-T',
 'rs4517468-T',
 'rs962369-C',
 'rs1004446-G',
 'rs1003484-G',
 'rs4520-C',
 'rs757158-T',
 'rs2230808-A',
 'rs3890182-A',
 'rs4149268-C',
 'rs8114057-A',
 'rs769455-C',
 'rs449647-A',
 'rs440446-G',
 'rs174575-G',
 'rs17831757-A',
 'rs6008259-A',
 'rs151216-C',
 'rs626283-C',
 'rs10416555-G',
 'rs11084313-T',
 'rs6756629-G',
 'rs4245791-C'

In [235]:
ambiguity_list = []
for i in rsid_list:
    rsid_mask = df_merged_drop_less_gwa['SNPS'].str.contains(i)
    df_merged_drop_less_gwa_rsid = df_merged_drop_less_gwa[rsid_mask].drop_duplicates()
    risk_allele = df_merged_drop_less_gwa_rsid['STRONGEST SNP-RISK ALLELE'].value_counts()#.index[0]
    if len(risk_allele)>1:
        if risk_allele[0] == risk_allele[1]:
            ambiguity = risk_allele.index[0], risk_allele[0]
            ambiguity_list.append(ambiguity)
            #print(risk_allele)
ambiguity_df = pd.DataFrame(ambiguity_list)

In [231]:
#ambiguity_df.to_csv('ambiguity_magg1_df.csv')
pd.read_csv('ambiguity_1_list.csv')

,Unnamed: 0,SNPS
0,0,rs10852521-A
1,1,rs11075994-A
2,2,rs2072518-A
3,3,rs9651118-A
4,4,rs1010023-C
...,...,...
244,244,rs60461843-T
245,245,rs561241-C
246,246,rs917911-C
247,247,rs10953541-C


In [236]:
ambiguity_df

,0,1
0,rs7185735-G,2
1,rs10852521-A,1
2,rs11075994-A,1
3,rs2072518-A,1
4,rs9651118-A,1
...,...,...
336,rs561241-C,1
337,rs917911-C,1
338,rs10953541-C,1
339,rs8017377-A,4


In [238]:
ambiguity_df.groupby(by=1).describe().to_clipboard()

### remove ambiguity

In [197]:
risk_allele_df#[0]
df_merged_drop_less_gwa
type(risk_allele_df.iloc[:,0])

pandas.core.series.Series

In [217]:
# select just ambiugous count > 1
ambiguous_magg1 = pd.read_csv('ambiguity_magg1_df.csv', index_col=0)
# Only GWAS
mask = df_merged_drop_less_gwa['SNPS'].isin(ambiguous_magg1.rsid)
df_merged_drop_less_gwa_ambmagg1 = df_merged_drop_less_gwa[mask]
df_merged_drop_less_gwa_ambmagg1#.to_csv('df_merged_drop_less_gwa_ambmagg1.csv')

,MAPPED_GENE,SNPS,DISEASE/TRAIT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
101759,FTO,rs7185735,Obesity,rs7185735-G,0.4
101760,FTO,rs7185735,Subcutaneous adipose tissue,rs7185735-A,0.58
101761,FTO,rs7185735,Pulse pressure x alcohol consumption interacti...,rs7185735-A,0.59
101762,FTO,rs7185735,Type 2 diabetes,rs7185735-G,0.397306062260211
816977,PPARG,rs11709077,Type 2 diabetes,rs11709077-G,0.8765
...,...,...,...,...,...
4638611,NYNRIN,rs8017377,Total cholesterol levels,rs8017377-G,NR
4638612,NYNRIN,rs8017377,Low density lipoprotein cholesterol levels,rs8017377-G,0.533
4638613,NYNRIN,rs8017377,Low density lipoprotein cholesterol levels,rs8017377-G,NR
4638614,NYNRIN,rs8017377,Total cholesterol levels,rs8017377-A,0.32


In [219]:
#GRPMX-GWAS
mask = df_merged_drop_nonan['SNPS'].isin(ambiguous_magg1.rsid)
df_merged_drop_nonan_ambmagg1 = df_merged_drop_nonan[mask]
df_merged_drop_nonan_ambmagg1[['LITVAR GENE','PUBMED MESH','DISEASE/TRAIT','STRONGEST SNP-RISK ALLELE','RISK ALLELE FREQUENCY','MAPPED_GENE']]#.to_csv('df_merged_drop_less_gwa_ambmagg1.csv')

,LITVAR GENE,PUBMED MESH,DISEASE/TRAIT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY,MAPPED_GENE
101759,FTO,Coronary Artery Disease,Obesity,rs7185735-G,0.4,FTO
101760,FTO,Coronary Artery Disease,Subcutaneous adipose tissue,rs7185735-A,0.58,FTO
101761,FTO,Coronary Artery Disease,Pulse pressure x alcohol consumption interacti...,rs7185735-A,0.59,FTO
101762,FTO,Coronary Artery Disease,Type 2 diabetes,rs7185735-G,0.397306062260211,FTO
101763,FTO,Depression,Obesity,rs7185735-G,0.4,FTO
...,...,...,...,...,...,...
4638710,NYNRIN,Triglycerides,Total cholesterol levels,rs8017377-G,NR,NYNRIN
4638711,NYNRIN,Triglycerides,Low density lipoprotein cholesterol levels,rs8017377-G,0.533,NYNRIN
4638712,NYNRIN,Triglycerides,Low density lipoprotein cholesterol levels,rs8017377-G,NR,NYNRIN
4638713,NYNRIN,Triglycerides,Total cholesterol levels,rs8017377-A,0.32,NYNRIN


In [221]:
df_merged_drop_nonan[['LITVAR GENE','PUBMED MESH','DISEASE/TRAIT','STRONGEST SNP-RISK ALLELE','RISK ALLELE FREQUENCY','MAPPED_GENE']]
#quanti sono monorisk allele???

,LITVAR GENE,PUBMED MESH,DISEASE/TRAIT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY,MAPPED_GENE
0,FTO,Eating,Dietary macronutrient intake,rs1421085-C,0.42,FTO
1,FTO,Eating,Obesity,rs1421085-C,0.4,FTO
2,FTO,Eating,Obesity,rs1421085-C,0.4,FTO
3,FTO,Eating,Obesity,rs1421085-C,0.41,FTO
4,FTO,Eating,Obesity (early onset extreme),rs1421085-C,0.41,FTO
...,...,...,...,...,...,...
4639187,ALOX12,Obesity,Height,rs434473-G,0.4008,"ALOX12, ALOX12-AS1"
4639188,ALOX12,Body Fat Distribution,Height,rs312462-A,0.0985,"ALOX12-AS1, ALOX12"
4639189,ALOX12,Body Mass Index,Height,rs312462-A,0.0985,"ALOX12-AS1, ALOX12"
4639190,ALOX12,Bone Density,Height,rs312462-A,0.0985,"ALOX12-AS1, ALOX12"


In [250]:
# before filter mother df with "riskallelelist

# Only GWAS
mask = df_merged_drop_less_gwa['STRONGEST SNP-RISK ALLELE'].isin(risk_allele_df.iloc[:,0])
df_merged_drop_less_gwa_riskall = df_merged_drop_less_gwa[mask]
#df_merged_drop_less_gwa_riskall.to_csv(r'C:\Users\Public\Database_SSD\GWAS calalog data\df_merged_drop_less_gwa_riskall.csv')
df_merged_drop_less_gwa_riskall

,MAPPED_GENE,SNPS,DISEASE/TRAIT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
0,FTO,rs1421085,Dietary macronutrient intake,rs1421085-C,0.42
1,FTO,rs1421085,Obesity,rs1421085-C,0.4
3,FTO,rs1421085,Obesity,rs1421085-C,0.41
4,FTO,rs1421085,Obesity (early onset extreme),rs1421085-C,0.41
6,FTO,rs1421085,Childhood body mass index,rs1421085-C,0.41
...,...,...,...,...,...
4639145,ADCY9,rs2444217,Height,rs2444217-A,0.25059
4639158,ADCY9,rs1045475,Appendicular lean mass,rs1045475-A,0.1782
4639159,HROB,rs227584,Bone mineral density (hip),rs227584-T,0.73
4639183,"ALOX12, ALOX12-AS1",rs434473,Height,rs434473-G,0.4008


In [203]:
# GRPMX-GWAS merged
mask2 = df_merged_drop_nonan['STRONGEST SNP-RISK ALLELE'].isin(risk_allele_df.iloc[:,0])
df_merged_drop_nonan_riskall = df_merged_drop_nonan[mask2]
df_merged_drop_nonan_riskall.to_csv(r'C:\Users\Public\Database_SSD\GWAS calalog data\df_merged_drop_nonan_riskall.csv')

In [264]:
# then remove ambiugous rsids

# GRPMX-GWAS merged
ambiguous_rsids = pd.read_csv('ambiguous_rsids.csv')
mask_amb = df_merged_drop_nonan_riskall.SNPS.isin(ambiguous_rsids['ambiguous_rsids'])
df_merged_drop_nonan_riskall_unamb = df_merged_drop_nonan_riskall[-mask_amb]

In [267]:
df_merged_drop_nonan_riskall_unamb.columns

Index(['LITVAR GENE', 'LIVAR RSID', 'LITVAR PMID', 'PUBMED MESH', 'PUBMEDID',
       'FIRST AUTHOR', 'DATE', 'JOURNAL', 'LINK', 'STUDY', 'DISEASE/TRAIT',
       'INITIAL SAMPLE SIZE', 'REPLICATION SAMPLE SIZE', 'REGION', 'CHR_ID',
       'CHR_POS', 'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'CONTEXT', 'INTERGENIC', 'RISK ALLELE FREQUENCY', 'P-VALUE',
       'PVALUE_MLOG', 'P-VALUE (TEXT)', 'OR or BETA', '95% CI (TEXT)',
       'PLATFORM [SNPS PASSING QC]', 'CNV', 'MAPPED_TRAIT', 'MAPPED_TRAIT_URI',
       'STUDY ACCESSION', 'GENOTYPING TECHNOLOGY'],
      dtype='object')

In [272]:
df_merged_drop_nonan_riskall_unamb[['LITVAR GENE', 'LIVAR RSID', 'LITVAR PMID', 'PUBMED MESH', 'PUBMEDID', 'DISEASE/TRAIT','MAPPED_GENE','STRONGEST SNP-RISK ALLELE', 'SNPS','RISK ALLELE FREQUENCY', 'P-VALUE','OR or BETA']][300:325].to_clipboard(sep=',')

In [251]:
# Only GWAS
mask_amb = df_merged_drop_less_gwa_riskall.SNPS.isin(ambiguous_rsids['ambiguous_rsids'])
df_merged_drop_less_gwa_riskall_unamb = df_merged_drop_less_gwa_riskall[-mask_amb]
df_merged_drop_less_gwa_riskall_unamb.to_csv(r'C:\Users\Public\Database_SSD\GWAS calalog data\df_merged_drop_less_gwa_unambiguous.csv')

In [261]:
df_merged_drop_less_gwa_riskall_unamb[300:325].to_clipboard(sep=',')

In [174]:
len(df_merged_drop_less_gwa_riskall_unamb.SNPS.drop_duplicates()), len(df_merged_drop_nonan.SNPS.drop_duplicates())
unambiguous_rsids = df_merged_drop_less_gwa_riskall_unamb.SNPS.drop_duplicates().reset_index(drop=True)
#unambiguous_rsids.to_csv('unambiguous_rsids.csv')
#df_merged_drop_nonan_unamb.to_csv(r'C:\Users\Public\Database_SSD\GWAS calalog data\df_merged_drop_nonan_unambiguous.csv')

In [178]:
df_merged_drop_less_gwa_unamb

,MAPPED_GENE,SNPS,DISEASE/TRAIT,STRONGEST SNP-RISK ALLELE,RISK ALLELE FREQUENCY
0,FTO,rs1421085,Dietary macronutrient intake,rs1421085-C,0.42
1,FTO,rs1421085,Obesity,rs1421085-C,0.4
3,FTO,rs1421085,Obesity,rs1421085-C,0.41
4,FTO,rs1421085,Obesity (early onset extreme),rs1421085-C,0.41
6,FTO,rs1421085,Childhood body mass index,rs1421085-C,0.41
...,...,...,...,...,...
4639144,ADCY9,rs2444217,Body mass index,rs2444217-A,0.57
4639145,ADCY9,rs2444217,Height,rs2444217-A,0.25059
4639158,ADCY9,rs1045475,Appendicular lean mass,rs1045475-A,0.1782
4639183,"ALOX12, ALOX12-AS1",rs434473,Height,rs434473-G,0.4008


In [67]:
# LOOKUP FOR SINGLE RSID
rsid_mask = df_merged_drop_less_gwa['SNPS'].str.contains('rs1558902')
df_merged_drop_less_gwa_rsid = df_merged_drop_less_gwa[rsid_mask]
df_merged_drop_less_gwa_rsid.to_clipboard()

In [ ]:
# LOOKUP FOR SINGLE ALLELE
allele_mask = df_merged_drop_nonan_rsid['STRONGEST SNP-RISK ALLELE'].str.contains('-C')
df_merged_drop_rsid_allele = df_merged_drop_nonan_rsid[allele_mask]
df_merged_drop_rsid_allele[['LITVAR GENE','LIVAR RSID','MAPPED_GENE','PUBMED MESH', 'DISEASE/TRAIT', 'STRONGEST SNP-RISK ALLELE', 'RISK ALLELE FREQUENCY']].drop_duplicates()

In [ ]:
dfx = pd.read_csv('grpm_survey/'+gene2+'_filtered_pmidrsidmesh.csv').drop(columns=['Unnamed: 0'])
gene2_pmidmesh = pd.read_csv('grpm_db/'+gene2+'_pubmed_pmidmesh.csv').drop(columns=['Unnamed: 0'])
dfy = gene2_pmidmesh[gene2_pmidmesh.pmids.isin(dfx.pmids)]

dfxy = pd.merge(dfx, dfy, left_on= '')